### You can run the following chunk of code to import any libraries and packages required to run the rest of the script.

In [1]:
import os as os
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv2
import tensorflow as tf
from scipy.ndimage import label
from skimage import measure
import pandas as pd

### You can run the following chunk of code to count cells in a set of MBM images or MBM video frames, using a size threshold.

The user will need to adjust the base_Directory variable, and carefully note the directory structure when using this code for your own purposes.

In [12]:
base_Directory = ""

output_Data_Frame = pd.DataFrame(columns = ["File Name", "Counts"])
channel_Directory = base_Directory + "/Analysis/All_Images_For_Analysis/"
network_File_Path = base_Directory + "/Training/Phase_1/Trained_Model/Phase_One_Trained_Network.h5"

network = tf.keras.models.load_model(network_File_Path)

def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

# Optimal sRBC Threshold
threshold = [90]

#Optimal Car-T Threshold
#threshold = [40]

for image_Name in os.listdir(channel_Directory):
    print("Analyzing " + image_Name[:-4])
    full_Channel = plt.imread(channel_Directory + image_Name)
    if np.max(full_Channel) == int(np.max(full_Channel)) and len(str(np.max(full_Channel))) == len(str(int(np.max(full_Channel)))):
        full_Channel = full_Channel.copy()/255.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel.copy(), cv2.COLOR_GRAY2RGB)
    if np.shape(full_Channel)[2] == 4:
        full_Channel = full_Channel.copy()[:,:,0:3]
    image_Height, image_Width, channels = np.shape(full_Channel)
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))
    x_Slider = 0
    y_Slider = 0
    output_Array = np.zeros((128,128))
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            output = network.predict(current_Tile_Normalized, verbose = 0)

            for i in range(128):
                for j in range(128):
                    output_Array[i,j] = np.argmax(output[0,i,j,:])
            
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    for i in range(image_Height_Resized):
        for j in range(image_Width_Resized):
            if output_Image[i,j] != 0:
                output_Image[i,j] = 1
            else:
                continue
    blobs, number_Of_Blobs = label(output_Image)
    properties = measure.regionprops(blobs)
    for thresh in threshold:
        centroids = [prop.centroid for prop in properties if prop.area > thresh]
        #output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids)]], columns = ["File Name", "Counts"]))
        output_Data_Frame = pd.concat([output_Data_Frame.copy(), pd.DataFrame([[image_Name,len(centroids)]], columns = ["File Name", "Counts"])])
    display(output_Data_Frame)
    print("======================================")

Analyzing Frame_1


,File Name,Counts
0,Frame_1.png,37


Analyzing Frame_10


,File Name,Counts
0,Frame_1.png,37
0,Frame_10.png,33


Analyzing Frame_100


...

,File Name,Counts
0,Frame_1.png,37
0,Frame_10.png,33
0,Frame_100.png,37
0,Frame_101.png,39
0,Frame_102.png,37
0,Frame_103.png,37
0,Frame_104.png,39


Analyzing Frame_105


,File Name,Counts
0,Frame_1.png,37
0,Frame_10.png,33
0,Frame_100.png,37
0,Frame_101.png,39
0,Frame_102.png,37
0,Frame_103.png,37
0,Frame_104.png,39
0,Frame_105.png,41


Analyzing Frame_106


KeyboardInterrupt: 

### You can run the following chunk of code to save the counts collected to a .csv file for further analysis, or for viewing.

In [11]:
output_Data_Frame.to_csv(base_Directory + "/Analysis/Automated_Counts/Counts_Test.csv")